In [ ]:
import os
#os.environ["JAX_PLATFORM_NAME"] = "cpu"

import jax_sbgeom as jsb
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
import jax
jax.config.update("jax_enable_x64", True)
import numpy as onp
import sys 
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)


from functools import partial
import jax_sbgeom.coils as jsc
import jax_sbgeom.flux_surfaces as jsf

from dataclasses import dataclass
import functools
from jax_sbgeom.jax_utils.utils import _mesh_to_pyvista_mesh, cumulative_trapezoid_uniform_periodic, interp1d_jax
import StellBlanket.SBGeom as sbg


import matplotlib.pyplot as plt
vmec_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/helias3_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/helias5_vmec.nc4", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_vmec.nc4"]
vmec_file = vmec_files[2]

coil_files = ["/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS3_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/HELIAS5_coils_all.h5", "/home/tbogaarts/stellarator_paper/base_data/vmecs/squid_coilset.h5"]

In [ ]:
fs_base = jsb.flux_surfaces.FluxSurface.from_hdf5(vmec_file)

In [ ]:

coilsd = onp.loadtxt("filament_s4uu_5ci_23")[:,:]
amax = 101
i = 0
coils = []
currents = []
for i in range(40):
#    if i not in [0, 4, 8, 12, 16, 20,24, 28, 32,35, 36,39] :
    coils.append(coilsd[i * amax : i * amax + amax -1,:-1])
    currents.append(coilsd[i * amax, -1])



jax_coils = [jsc.DiscreteCoil.from_positions(jnp.array(coil)) for coil in coils]
jax_coilset = jsc.CoilSet.from_list(jax_coils)
jax_currents = jnp.array(currents)

sbg_coilset = sbg.Coil_Set([sbg.Discrete_Coil(coils[i]) for i in range(len(coils))])

In [ ]:
coilset = jsc.fourier_coil.convert_to_fourier_coilset(jax_coilset)

coilset_resampled = jsc.fourier_coil.convert_fourier_coilset_to_equal_arclength(coilset)

In [ ]:
def test_converting_fourier_coils(_get_all_discrete_coils):
    coilset_jaxsbgeom, coilset_sbgeom = _get_all_discrete_coils
    fourier_coilset_sbgeom = sbg.Coils.Convert_to_Fourier_Coils(coilset_sbgeom)

    for i in range(coilset_sbgeom.Number_of_Coils()):
        coil_jsb = coilset_jaxsbgeom[i]
        fourier_coil_sbgeom = fourier_coilset_sbgeom[i]
        
        
        fourier_coil_jaxsbgeom = jsb.coils.fourier_coil.convert_to_fourier_coil(coil_jsb)

        onp.testing.assert_allclose(fourier_coil_jaxsbgeom.fourier_cos, fourier_coil_sbgeom.Get_Fourier_Cos(), rtol=1e-12, atol=1e-12)
        onp.testing.assert_allclose(fourier_coil_jaxsbgeom.fourier_sin, fourier_coil_sbgeom.Get_Fourier_Sin(), rtol=1e-12, atol=1e-12)
        print(fourier_coil_jaxsbgeom.centre)
        onp.testing.assert_allclose(fourier_coil_jaxsbgeom.centre_i, fourier_coil_sbgeom.Get_Centre(), rtol=1e-12, atol=1e-12)

test_converting_fourier_coils(_get_all_discrete_coils=(jax_coilset, sbg_coilset))

In [ ]:
s_samples = jnp.linspace(0.0, 1.0, 200, endpoint=False)
arclengths_vmap = jax.vmap(jsc.fourier_coil._arc_length_fourier, in_axes=(0,None))


arclengths_base = arclengths_vmap(coilset.coils, s_samples)
arclengths_resampled = arclengths_vmap(coilset_resampled.coils, s_samples)


In [ ]:
base_arclength = plt.plot(s_samples, arclengths_base.T)
plt.figure()
resampled_arclength = plt.plot(s_samples, arclengths_resampled.T)

In [ ]:
jp